In [1]:
import os
%pwd

'/Users/hh/MLops/nlp_project/research'

In [2]:
os.chdir("../")
%pwd

'/Users/hh/MLops/nlp_project'

In [3]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [4]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories
class ConfigurationManager():
    def __init__(self, config_path=CONFIG_FILE_PATH, params_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        create_directories([config.root_dir])
        model_evaluatin_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_name = config.metric_file_name
        )
        return model_evaluatin_config

In [12]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_from_disk
import torch
import pandas as pd
from tqdm import tqdm
import evaluate

In [17]:
class ModelEvaluation:
    def __init__(self, model_evaluation_config:ModelEvaluationConfig):
        self.config = model_evaluation_config
    
    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer,
        batch_size=16, device="cpu",
        column_text="article",
        column_summary="highlights"):

        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):

            inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                        attention_mask=inputs["attention_mask"].to(device),
                                        length_penalty=0.8, num_beams=8, max_length=128)

            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                        clean_up_tokenization_spaces=True)
                    for s in summaries]

            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

            metric.add_batch(predictions=decoded_summaries, references=target_batch)

        score = metric.compute()
        return score
    
    def evaluate(self):
        device = "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

        dataset_samsum_pt = load_from_disk(self.config.data_path)
        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
        rouge_metric = evaluate.load('rouge')

        score = self.calculate_metric_on_test_ds(
            dataset=dataset_samsum_pt['test'][0:10], metric=rouge_metric, model=model_pegasus, tokenizer=tokenizer, batch_size=2, column_text="dialogue", column_summary="summary"
        )

        rouge_dict = {rn: score[rn] for rn in rouge_names}

        df = pd.DataFrame(rouge_dict, index=['pegasus'])
        df.to_csv(self.config.metric_file_name, index=False)




In [18]:
manager = ConfigurationManager()
config = manager.get_model_evaluation_config()
obj = ModelEvaluation(config)
obj.evaluate()


[2025-12-16 23:41:00,524] 13 src.textSummarizer.logging - INFO - entered read_yaml function at /Users/hh/MLops/nlp_project
[2025-12-16 23:41:00,527] 16 src.textSummarizer.logging - INFO - yaml file: config/config.yaml loaded
[2025-12-16 23:41:00,528] 13 src.textSummarizer.logging - INFO - entered read_yaml function at /Users/hh/MLops/nlp_project
[2025-12-16 23:41:00,529] 16 src.textSummarizer.logging - INFO - yaml file: params.yaml loaded
[2025-12-16 23:41:00,529] 29 src.textSummarizer.logging - INFO - created directory at artifacts
[2025-12-16 23:41:00,530] 29 src.textSummarizer.logging - INFO - created directory at artifacts/model_evaluation


100%|██████████| 5/5 [02:07<00:00, 25.48s/it]

[2025-12-16 23:43:08,874] 83 absl - INFO - Using default tokenizer.
